# Homework and bake-off: pragmatic color descriptions

In [133]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2021"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [All two-word examples as a dev corpus](#All-two-word-examples-as-a-dev-corpus)
1. [Dev dataset](#Dev-dataset)
1. [Random train–test split for development](#Random-train–test-split-for-development)
1. [Question 1: Improve the tokenizer [1 point]](#Question-1:-Improve-the-tokenizer-[1-point])
1. [Use the tokenizer](#Use-the-tokenizer)
1. [Question 2: Improve the color representations [1 point]](#Question-2:-Improve-the-color-representations-[1-point])
1. [Use the color representer](#Use-the-color-representer)
1. [Initial model](#Initial-model)
1. [Question 3: GloVe embeddings [1 point]](#Question-3:-GloVe-embeddings-[1-point])
1. [Try the GloVe representations](#Try-the-GloVe-representations)
1. [Question 4: Color context [3 points]](#Question-4:-Color-context-[3-points])
1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bakeoff [1 point]](#Bakeoff-[1-point])

## Overview

This homework and associated bake-off are oriented toward building an effective system for generating color descriptions that are pragmatic in the sense that they would help a reader/listener figure out which color was being referred to in a shared context consisting of a target color (whose identity is known only to the describer/speaker) and a set of distractors.

The notebook [colors_overview.ipynb](colors_overview.ipynb) should be studied before work on this homework begins. That notebook provides backgroud on the task, the dataset, and the modeling code that you will be using and adapting.

The homework questions are more open-ended than previous ones have been. Rather than asking you to implement pre-defined functionality, they ask you to try to improve baseline components of the full system in ways that you find to be effective. As usual, this culminates in a prompt asking you to develop a novel system for entry into the bake-off. In this case, though, the work you do for the homework will likely be directly incorporated into that system (not required, but an efficient way to work at the very least).

## Set-up

See [colors_overview.ipynb](colors_overview.ipynb) for set-up in instructions and other background details.

In [1]:
from colors import ColorsCorpusReader
from nltk.translate.bleu_score import corpus_bleu
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from torch_color_describer import ContextualColorDescriber
from torch_color_describer import create_example_dataset

import utils
from utils import START_SYMBOL, END_SYMBOL, UNK_SYMBOL

!pip install editdistance
if 'IS_GRADESCOPE_ENV' not in os.environ:
    !pip install pyspellchecker
    !pip install nltk
    !pip install python-Levenshtein
    !pip install fuzzywuzzy[speedup]
    !pip install fuzzywuzzy
    !pip install scipy

zsh:1: no matches found: fuzzywuzzy[speedup]


In [2]:
utils.fix_random_seeds()

In [3]:
COLORS_SRC_FILENAME = os.path.join("data", "colors", "filteredCorpus.csv")

## All two-word examples as a dev corpus

So that you don't have to sit through excessively long training runs during development, I suggest working with the two-word-only subset of the corpus until you enter into the late stages of system testing.

In [4]:
dev_corpus = ColorsCorpusReader(
    COLORS_SRC_FILENAME,
    word_count=2,
    normalize_colors=True)

In [5]:
dev_examples = list(dev_corpus.read())

This subset has about one-third the examples of the full corpus:

In [6]:
len(dev_examples)

13890

We __should__ worry that it's not a fully representative sample. Most of the descriptions in the full corpus are shorter, and a large proportion are longer. So this dataset is mainly for debugging, development, and general hill-climbing. All findings should be validated on the full dataset at some point.

## Dev dataset

The first step is to extract the raw color and raw texts from the corpus:

In [7]:
dev_rawcols, dev_texts = zip(*[[ex.colors, ex.contents] for ex in dev_examples])

The raw color representations are suitable inputs to a model, but the texts are just strings, so they can't really be processed as-is. Question 1 asks you to do some tokenizing!

## Random train–test split for development

For the sake of development runs, we create a random train–test split:

In [8]:
dev_rawcols_train, dev_rawcols_test, dev_texts_train, dev_texts_test = \
    train_test_split(dev_rawcols, dev_texts)

## Question 1: Improve the tokenizer [1 point]

This is the first required question – the first required modification to the default pipeline.

The function `tokenize_example` simply splits its string on whitespace and adds the required start and end symbols:

In [9]:
from typing import List
import string

SECOND_TRY_TOKEN = "###"

def get_best_replacement(word: str, min_fuzzy_match: int = 85) -> str:
    """
    Remove punctuation, look for closest matching 
    color name or just correct the spelling.
    """
    word = word.lower()
    word = word.translate(str.maketrans('', '', string.punctuation))
    if not word.isalpha():
        return ""
    if word == spell.correction(word):
        return word
    
    # find the closest words that are color names
    candidates = spell.candidates(word)
    for c, match in process.extract(word, color_names, limit=3):
        if match > min_fuzzy_match:
            candidates.add(c)

    color_cands = set(candidates) & color_names
    if not color_cands:
        return spell.correction(word)
    
    top_choice = next(iter(color_cands))
    return top_choice


def spell_check_words(words: List[str]) -> None:
    # in-place
    for i, word in enumerate(words):
        if word == SECOND_TRY_TOKEN:
            continue
        words[i] = get_best_replacement(word)

def tokenize_example(s, use_sound_tokens=False): 
    """
    Tokenize a string byt performing a best-effort 
    customized spellcheck and removing punctuation.
    """ 
    words = s.split()
    spell_check_words(words)
    sentence = " ".join(words)
    if use_sound_tokens:
        return [START_SYMBOL] + SSP.tokenize(sentence) + [END_SYMBOL]
    
    return [START_SYMBOL] + TT.tokenize(sentence) + [END_SYMBOL]

In [13]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    # eyeball test
    from random import sample
    import matplotlib.colors as mcolors
    from spellchecker import SpellChecker
    import editdistance
    from fuzzywuzzy import fuzz
    from fuzzywuzzy import process
    from nltk.tokenize import SyllableTokenizer
    from nltk.tokenize import TreebankWordTokenizer

    # sound based tokenizer
    SSP = SyllableTokenizer()
    # slighter better than space tokenizer
    TT = TreebankWordTokenizer()
    
    color_names = set([
        name 
        for names in mcolors.CSS4_COLORS 
        for name in names.split()
    ])
    color_names = color_names | {
        name
        for names in mcolors.XKCD_COLORS
        for name in names.replace("xkcd:", "").split()
    }
    
    spell = SpellChecker(language='en')
    print("Some Known Colors: ", sample(color_names, 5))

    tokenize_example(dev_texts_train[376])
    for text in sample(dev_texts_train + dev_texts_test, 20):
        print(text, tokenize_example(text))

Some Known Colors:  ['bubble', 'bruise', 'powderblue', 'hazel', 'pale']
sky blue ['<s>', 'sky', 'blue', '</s>']
hot red ['<s>', 'hot', 'red', '</s>']
bright green ['<s>', 'bright', 'green', '</s>']
camo green... ['<s>', 'camo', 'green', '</s>']
greyish one ['<s>', 'greyish', 'one', '</s>']
bright green ['<s>', 'bright', 'green', '</s>']
dull pink ['<s>', 'dull', 'pink', '</s>']
grey blu ['<s>', 'grey', 'blu', '</s>']
darker purple ['<s>', 'darker', 'purple', '</s>']
bright pink ['<s>', 'bright', 'pink', '</s>']
ice blue ['<s>', 'ice', 'blue', '</s>']
normal purple ['<s>', 'normal', 'purple', '</s>']
Breakfast juice ['<s>', 'breakfast', 'juice', '</s>']
muted blue ### not grey ['<s>', 'muted', 'blue', '#', '#', '#', 'not', 'grey', '</s>']
dusty rose ['<s>', 'dusty', 'rose', '</s>']
neon green ['<s>', 'neon', 'green', '</s>']
Blueish one ['<s>', 'blueish', 'one', '</s>']
bright green ['<s>', 'bright', 'green', '</s>']
brownish green ['<s>', 'brownish', 'green', '</s>']
richest purple ['<

__Your task__: Modify `tokenize_example` so that it does something more sophisticated with the input text. 

__Notes__:

* There are useful ideas for this in [Monroe et al. 2017](https://transacl.org/ojs/index.php/tacl/article/view/1142)
* There is no requirement that you do word-level tokenization. Sub-word and multi-word are options.
* This question can interact with the size of your vocabulary (see just below), and in turn with decisions about how to use `UNK_SYMBOL`.

__Important__: don't forget to add the start and end symbols, else the resulting models will definitely be terrible! The following test will check that your tokenizer has this property:

In [14]:
def test_tokenize_example(func):
    s = "A test string"
    result = func(s)
    assert all(isinstance(tok, str) for tok in result), \
        "The tokenizer must return a list of strings."
    assert result[0] == START_SYMBOL, \
        "The tokenizer must add START_SYMBOL as the first token."
    assert result[-1] == END_SYMBOL, \
        "The tokenizer must add END_SYMBOL as the final token."

In [15]:
if 'IS_GRADESCOPE_ENV' not in os.environ:

    from spellchecker import SpellChecker
    import matplotlib.colors as mcolors
    import editdistance
    from fuzzywuzzy import fuzz
    from fuzzywuzzy import process
    from nltk.tokenize import SyllableTokenizer
    from nltk.tokenize import TreebankWordTokenizer

    # sound based tokenizer
    SSP = SyllableTokenizer()
    # slighter better than space tokenizer
    TT = TreebankWordTokenizer()

    # get names of colors
    color_names = set([
        name 
        for names in mcolors.CSS4_COLORS 
        for name in names.split()
    ])
    color_names = color_names | {
        name
        for names in mcolors.XKCD_COLORS
        for name in names.replace("xkcd:", "").split()
    }
    for ch in string.ascii_lowercase:
        # cleanup
        if ch in color_names:
            color_names.remove(ch)

    spell = SpellChecker(language='en')
    test_tokenize_example(tokenize_example)
print("Test 1 passed")

Test 1 passed


## Use the tokenizer

Once the tokenizer is working, run the following cell to tokenize your inputs:

In [16]:
dev_seqs_train = [tokenize_example(s) for s in dev_texts_train]

dev_seqs_test = [tokenize_example(s) for s in dev_texts_test]

We use only the train set to derive a vocabulary for the model:

In [17]:
dev_vocab = sorted({w for toks in dev_seqs_train for w in toks})

dev_vocab += [UNK_SYMBOL]

It's important that the `UNK_SYMBOL` is included somewhere in this list. In test examples, words not seen in training will be mapped to `UNK_SYMBOL`. 

Conceptual note: If you model's vocab is the same as your train vocab, then `UNK_SYMBOL` will never be encountered during training, so it will be a random vector at test time.

In [18]:
len(dev_vocab)

861

## Question 2: Improve the color representations [1 point]

This is the second required pipeline improvement for the assignment. 

The following functions do nothing at all to the raw input colors we get from the corpus. 

In [67]:
from scipy.fft import fft, fftfreq


def represent_color_context(colors):
    
    #print(color)
    #return np.abs(fft(colors))
    return [represent_color(color) for color in colors]


def represent_color(color):
    #print('color', color)
    # Improve me!
    scaled = [ value * 100 for value in color]
    return np.abs(fft(scaled)).tolist()

In [68]:
represent_color_context(dev_rawcols_train[0])

[[80.44444444444444, 35.53836389934088, 35.53836389934088],
 [151.72222222222223, 41.33762671847587, 41.33762671847587],
 [150.22222222222223, 39.644874188004636, 39.644874188004636]]

__Your task__: Modify `represent_color_context` and/or `represent_color` to represent colors in a new way.
    
__Notes__:

* You are not required to keep `represent_color`. This might be unnatural if you want to perform an operation on each color trio all at once.
* For that matter, if you want to process all of the color contexts in the entire data set all at once, that is fine too, as long as you can also perform the operation at test time with an unknown number of examples being tested.

* The Fourier-transform method of [Monroe et al. 2016](https://www.aclweb.org/anthology/D16-1243/) and [Monroe et al. 2017](https://transacl.org/ojs/index.php/tacl/article/view/1142) is a proven choice for our task. __It is not required that you implement this.__ However, if you decide to, you might find that the overly terse presentation in the paper is an obstacle. They key thing to see is that the notation $\hat{f}_{jkl}$ is meant to specify a full coordinate system. Thus, you might do something like

```
from itertools import product
for j, k, l in product((0, 1, 2), repeat=3):    
    f_jkl = ...
```

and collect these `f_jkl` values in a list of 27 values. Additionally, in Python, [`2j` produces a value with `real` and `imag` attributes](https://docs.python.org/3.7/library/cmath.html). Each element `f_jkl` should have these components. If you concatenate the `real` and `imag` parts of all the `f_jkl`, you will have a 54-dimensional representation, as in the paper. Remember to start with an HSV representation, and with $h$ in $[0, 360]$, $s$ in $[0, 200]$, and $v$ in $[0, 200]$ (or else do the scaling differently). Note that the values in our corpus are in HLS format, [which are easily converted to HSV](https://en.wikipedia.org/wiki/HSL_and_HSV#HSV_to_HSL).

The following test seeks to ensure only that the output of your `represent_color_context` will be compatible with the models we are creating:

In [69]:
def test_represent_color_context(func):
    """`func` should be `represent_color_context`"""
    example = [
        [0.786, 0.58, 0.87],
        [0.689, 0.44, 0.92],
        [0.628, 0.32, 0.81]]
    result = func(example)
    assert len(result) == len(example), \
        ("Color context representations need to represent each color "
         "separately. (We assume the final color is the target.)")
    for i, color in enumerate(result):
        assert all(isinstance(x, float) for x in color), \
            ("All color representations should be lists of floats. "
             "Color {} is {}".format(i, color))

In [70]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_represent_color_context(represent_color_context)
print("Test 2 passed")

Test 2 passed


## Use the color representer

The following cell just runs your `represent_color_context` on the train and test sets:

In [71]:
dev_cols_train = [represent_color_context(colors) for colors in dev_rawcols_train]

dev_cols_test = [represent_color_context(colors) for colors in dev_rawcols_test]

At this point, our preprocessing steps are complete, and we can fit a first model.

## Initial model

The first model is configured right now to be a small model run for just a few iterations. It should be enough to get traction, but it's unlikely to be a great model. You are free to modify this configuration if you wish; it is here just for demonstration and testing:

In [72]:
dev_mod = ContextualColorDescriber(
    dev_vocab,
    early_stopping=True,
)

In [73]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    %time _ = dev_mod.fit(dev_cols_train, dev_seqs_train)
else:
    dev_mod.fit(dev_cols_train, dev_seqs_train)

Stopping after epoch 104. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 39.07379746437073

CPU times: user 9min 47s, sys: 9.76 s, total: 9min 57s
Wall time: 6min 33s


The canonical bake-off evaluation function is `evaluate`. Our primary metric is `listener_accuracy`; the BLEU score is included as a check to ensure that your system is speaking English!

In [74]:
evaluation = dev_mod.evaluate(dev_cols_test, dev_seqs_test)

In [75]:
evaluation.keys()

dict_keys(['listener_accuracy', 'corpus_bleu', 'target_index', 'predicted_index', 'predicted_utterance'])

In [76]:
evaluation['listener_accuracy']

0.6210768787791535

In [77]:
dev_mod.listener_accuracy(dev_cols_test, dev_seqs_test)

0.6210768787791535

In [78]:
evaluation['corpus_bleu']

0.6060389583278525

In [79]:
bleu, predicted_utterances = dev_mod.corpus_bleu(dev_cols_test, dev_seqs_test)

bleu

0.6060389583278525

In [80]:
evaluation['target_index'][: 5]

[2, 2, 2, 2, 2]

In [81]:
evaluation['predicted_index'][: 5]

[0, 0, 2, 2, 2]

In [90]:
evaluation['predicted_utterance'][: 5]

[['<s>', 'bright', 'purple', '</s>'],
 ['<s>', 'yellow', 'green', '</s>'],
 ['<s>', 'dull', 'purple', '</s>'],
 ['<s>', 'bright', 'purple', '</s>'],
 ['<s>', 'bright', 'green', '</s>']]

We can also see the model's predicted sequences given color context inputs:

In [91]:
dev_mod.predict(dev_cols_test[: 1])

[['<s>', 'bright', 'purple', '</s>']]

In [92]:
dev_seqs_test[: 1]

[['<s>', 'right', 'side', '#', '#', '#', 'purple', 'pinkish', '</s>']]

## Question 3: GloVe embeddings [1 point]

The above model uses a random initial embedding, as configured by the decoder used by `ContextualColorDescriber`. This homework question asks you to consider using GloVe inputs. 

__Your task__: Complete `create_glove_embedding` so that it creates a GloVe embedding based on your model vocabulary. This isn't mean to be analytically challenging, but rather just to create a basis for you to try out other kinds of rich initialization.

In [171]:
GLOVE_HOME = os.path.join('data', 'glove.6B')

In [172]:
def create_glove_embedding(vocab, glove_base_filename='glove.6B.50d.txt'):
    # Read in the GloVe file
    word_vectors = utils.glove2dict(os.path.join(GLOVE_HOME, glove_base_filename))

    # Create the embedding.
    # This function will, by default, ensure that START_TOKEN,
    # END_TOKEN, and UNK_TOKEN are included in the embedding.
    embedding, vocab = utils.create_pretrained_embedding(word_vectors, vocab)

    # Return the embedding created as well as the vocabulary
    # which is likely to have been modified from the input `vocab`.
    return embedding, vocab



In [173]:
def test_create_glove_embedding(func):
    vocab = ['NLU', 'is', 'the', 'future', '.', '$UNK', '<s>', '</s>']
    glove_embedding, glove_vocab = func(vocab, 'glove.6B.50d.txt')
    assert isinstance(glove_embedding, np.ndarray), \
        "Expected embedding type {}; got {}".format(
        glove_embedding.__class__.__name__, glove_embedding.__class__.__name__)
    assert glove_embedding.shape == (8, 50), \
        "Expected embedding shape (8, 50); got {}".format(glove_embedding.shape)
    assert glove_vocab == vocab, \
        "Expected vocab {}; got {}".format(vocab, glove_vocab)

In [174]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_create_glove_embedding(create_glove_embedding)
print("Test 3 passed")

Test 3 passed


## Try the GloVe representations

The extent to which GloVe is useful will depend heavily on how aligned your tokenization scheme is with the GloVe vocabulary. For example, if you did character-level tokenization, then the GloVe embedding space is not well-aligned with your tokenizer and using GloVe should have little no positive effect.

Let's see if GloVe helped for our development data:

In [175]:
dev_glove_embedding, dev_glove_vocab = create_glove_embedding(dev_vocab)

In [176]:
len(dev_vocab)

912

In [177]:
len(dev_glove_vocab)

912

In [219]:
dev_mod_glove = ContextualColorDescriber(
    dev_glove_vocab,
    embedding=dev_glove_embedding,
    early_stopping=True,
)

In [179]:
_ = dev_mod_glove.fit(dev_cols_train, dev_seqs_train)

Stopping after epoch 12. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 53.69252061843872

In [180]:
dev_mod_glove.listener_accuracy(dev_cols_test, dev_seqs_test)

/home/jovyan/work/torch_color_describer.py:679: RuntimeWarning: divide by zero encountered in power
  perp = [np.prod(s)**(-1/len(s)) for s in scores]


0.3723006046645551

You probably saw a small boost, assuming your tokeization scheme leads to good overlap with the GloVe vocabulary. The input representations are larger than in our previous model (at least as I configured things), so we would need to do more runs with higher `max_iter` values to see whether this is worthwhile overall.

## Question 4: Color context [3 points]

The final required homework question is the most challenging, but it should set you up to think in much more flexible ways about the underlying model we're using.

The question asks you to modify various model components in `torch_color_describer.py`. The section called [Modifying the core model](colors_overview.ipynb#Modifying-the-core-model) from the core unit notebook provides a number of examples illustrating the basic techniques, so you might review that material if you get stuck here.

__Your task__: Building on ideas from [Monroe et al. 2017](https://transacl.org/ojs/index.php/tacl/article/view/1142), you will redesign the model so that the target color (the final one in the context) is appended to each input token that gets processed by the decoder. The question asks you to subclass the `Decoder` and `EncoderDecoder` from `torch_color_describer.py` so that you can build models that do this.

__Step 1__: Modify the `Decoder` so that the input vector to the model at each timestep is not just a token representation `x` but the concatenation of `x` with the representation of the target color.

__Notes__:

* You might notice at this point that the original `Decoder.forward` method has an optional keyword argument `target_colors` that is passed to `Decoder.get_embeddings`. Because this is already in place, all you have to do is modify the `get_embeddings` method to use this argument.

* The change affects the configuration of `self.rnn`, so you need to subclass the `__init__` method as well, so that its `input_size` argument accomodates the embedding as well as the color representations.

* You can do the relevant operations efficiently in pure PyTorch using `repeat_interleave` and `cat`, but the important thing is to get a working implementation – you can always optimize the code later if the ideas prove useful to you. 

Here's skeleton code for you to flesh out:

In [299]:
from torch_color_describer import Decoder
import torch
import torch.nn as nn


class ColorContextDecoder(Decoder):
    def __init__(self, color_dim, *args, **kwargs):
        self.color_dim = color_dim
        super().__init__(*args, **kwargs)

        # input vectors dimension is the sum of 1D embedding vector
        # dimension and the 1D color vector dimension.
        self.input_size = self.embed_dim + self.color_dim

        # Update the `self.rnn` attribute for the new input size
        self.rnn = nn.GRU(
            input_size=self.input_size,
            hidden_size=self.hidden_dim,
            batch_first=True,
        )


    def get_embeddings(self, word_seqs, target_colors=None):
        """
        Attach each color vector i to each of the tokens in
        the ith sequence of the embedded version of word_seqs.

        Params:
            word_seqs: array of indices of words in embedding matrix.
            target_colors: tensor of shape (m, n), where m is the 
                           length of the batch (same as `word_seqs.shape[0]`) 
                           and n is the dimensionality of the color representations 
                           the model is using.
        """
        word_only_embeddings = self.embedding(word_seqs)
        _, batch_len, _ = word_only_embeddings.shape  # no. of tokens in batch
        
        colors = target_colors.unsqueeze(1)  # convert each row to a column
        colors = torch.repeat_interleave(colors, repeats=batch_len, dim=1)
        return torch.cat(tensors=[word_only_embeddings, colors], dim=2)




Step 1 is the most demanding of the steps in terms of tensor wrangling. It's important to have a clear idea of what you are trying to achieve and to unit test `get_embeddings` so that you can check that it has realized your vision. The following test should help with that:

In [300]:
def test_get_embeddings(decoder_class):
    """
    It's assumed that the input to this will be `ColorContextDecoder`.
    You pass in the class, and the function initalizes it with the test
    parameters.
    """
    dec = decoder_class(
        color_dim=3,   # For these, we mainly want *different*
        vocab_size=10, # dimensions so that we reliably get
        embed_dim=4,   # dimensionality errors if something
        hidden_dim=5)  # isn't working.

    # This step just changes the embedding to one with values
    # that are easy to inspect and definitely will not change
    # between runs:
    dec.embedding = nn.Embedding.from_pretrained(
        torch.FloatTensor([
            [10, 11, 12, 13],
            [14, 15, 16, 17],
            [18, 19, 20, 21]]))

    # These are the incoming sequences -- lists of indices
    # into the rows of `dec.embedding`:
    word_seqs = torch.tensor([
        [0,1,2],
        [2,0,1]])

    # Target colors as small floats that will be easy to track:
    target_colors = torch.tensor([
        [0.1, 0.2, 0.3],
        [0.7, 0.8, 0.9],
    ])

    # The desired return value: one list of tensors for each of
    # the two sequences in `word_seqs`. Each index is replaced
    # with its vector from `dec.embedding` and has the
    # corrresponding color from `target_colors` appended to it.
    expected = torch.tensor([
        [[10., 11., 12., 13.,  0.1,  0.2,  0.3],
         [14., 15., 16., 17.,  0.1,  0.2,  0.3],
         [18., 19., 20., 21.,  0.1,  0.2,  0.3]],

        [[18., 19., 20., 21.,  0.7,  0.8,  0.9],
         [10., 11., 12., 13.,  0.7,  0.8,  0.9],
         [14., 15., 16., 17.,  0.7,  0.8,  0.9]],
    ])

    result = dec.get_embeddings(word_seqs, target_colors=target_colors)

    assert expected.shape == result.shape, \
        "Expected shape {}; got shape {}".format(expected.shape, result.shape)

    assert torch.all(expected.eq(result)), \
        ("Your result has the desired shape but the values aren't correct. "
         "Here's what your function creates; compare it with `expected` "
         "from the test:\n{}".format(result))

In [301]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_get_embeddings(ColorContextDecoder)
print("Test 4 Passed")

Test 4 Passed


__Step 2__: Modify the `EncoderDecoder`. For this, you just need to make a small change to the `forward` method: extract the target colors from `color_seqs` and feed them to the decoder.

In [315]:
from torch_color_describer import EncoderDecoder

class ColorizedEncoderDecoder(EncoderDecoder):

    def forward(
        self,
        color_seqs,
        word_seqs,
        seq_lengths=None,
        hidden=None,
        targets=None,
    ):
        if hidden is None:
            hidden = self.encoder(color_seqs)

        # Extract the target colors from `color_seqs` and
        # feed them to the decoder, which already has a
        # `target_colors` keyword
        output, hidden = self.decoder(
            word_seqs, 
            seq_lengths=seq_lengths, 
            hidden=hidden, 
            # the last color is always the target color for all samples.
            target_colors=color_seqs[:, -1, :],  
        )

        # Your decoder will return `output, hidden` pairs; the
        # following will handle the two return situations that
        # the code needs to consider -- training and prediction.
        if self.training:
            return output
        else:
            return output, hidden

__Step 3__: Finally, as in the examples in [Modifying the core model](colors_overview.ipynb#Modifying-the-core-model), you need to modify the `build_graph` method of `ContextualColorDescriber` so that it uses your new `ColorContextDecoder` and `ColorizedEncoderDecoder`. Here's starter code:

In [316]:
from torch_color_describer import Encoder

class ColorizedInputDescriber(ContextualColorDescriber):

    def build_graph(self):

        # We didn't modify the encoder, so this is
        # just copied over from the original.
        encoder = Encoder(
            color_dim=self.color_dim,
            hidden_dim=self.hidden_dim,
        )

        # Use the ColorContextDecoder, making sure
        # to pass in all the keyword arguments coming
        # from ColorizedInputDescriber.
        decoder = ColorContextDecoder(
            color_dim=self.color_dim,
            vocab_size=self.vocab_size,
            embed_dim=self.embed_dim,
            embedding=self.embedding,
            hidden_dim=self.hidden_dim,
        )


        # Return a `ColorizedEncoderDecoder` that uses
        # the custom encoder and decoders
        return ColorizedEncoderDecoder(encoder, decoder)



That's it! Since these modifications are pretty intricate, you might want to use [a toy dataset](colors_overview.ipynb#Toy-problems-for-development-work) to debug it:

In [317]:
def test_full_system(describer_class):
    toy_color_seqs, toy_word_seqs, toy_vocab = create_example_dataset(
        group_size=50, vec_dim=2)

    toy_color_seqs_train, toy_color_seqs_test, toy_word_seqs_train, toy_word_seqs_test = \
        train_test_split(toy_color_seqs, toy_word_seqs)

    toy_mod = describer_class(toy_vocab)

    _ = toy_mod.fit(toy_color_seqs_train, toy_word_seqs_train)

    acc = toy_mod.listener_accuracy(toy_color_seqs_test, toy_word_seqs_test)

    return acc

In [318]:
test_full_system(ColorizedInputDescriber)
print("Test 5 passed")

Finished epoch 1000 of 1000; error is 0.11974356323480606Test 5 passed


If that worked, then you can now try this model on SCC problems!

## Your original system [3 points]

There are many options for your original system, which consists of the full pipeline – all preprocessing and modeling steps. You are free to use any model you like, as long as you subclass `ContextualColorDescriber` in a way that allows its `evaluate` method to behave in the expected way.

So that we can evaluate models in a uniform way for the bake-off, we ask that you modify the function `evaluate_original_system` below so that it accepts a trained instance of your model and does any preprocessing steps required by your model.

If we seek to reproduce your results, we will rerun this entire notebook. Thus, it is fine if your `evaluate_original_system` makes use of functions you wrote or modified above this cell.

In [338]:
def evaluate_original_system(trained_model, color_seqs_test, texts_test):
    """
    Feel free to modify this code to accommodate the needs of
    your system. Just keep in mind that it will get raw corpus
    examples as inputs for the bake-off.

    """
    # `word_seqs_test` is a list of strings, so tokenize each of
    # its elements:
    tok_seqs = [tokenize_example(s) for s in texts_test]

    col_seqs = [represent_color_context(colors)
                for colors in color_seqs_test]


    # Optionally include other preprocessing steps here. Note:
    # DO NOT RETRAIN YOUR MODEL AS PART OF THIS EVALUATION!
    # It's a tempting step, but it's a mistake and will get
    # you disqualified!

    # The following core score calculations are required:
    evaluation = trained_model.evaluate(col_seqs, tok_seqs)

    return evaluation

If `evaluate_original_system` works on test sets you create from the corpus distribution, then it will work for the bake-off, so consider checking that. For example, this would check that `dev_mod` above passes muster:

In [339]:
my_evaluation = evaluate_original_system(dev_mod, dev_rawcols_test, dev_texts_test)

/home/jovyan/work/torch_color_describer.py:679: RuntimeWarning: divide by zero encountered in power
  perp = [np.prod(s)**(-1/len(s)) for s in scores]


In [340]:
my_evaluation['listener_accuracy']

0.38237834725021597

In [341]:
my_evaluation['corpus_bleu']

0.05000000000000001

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies. We also ask that you report the best **listener_accuracy** score your system got during development, just to help us understand how systems performed overall.

In [346]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    # use larger more expressive glove embeddings
    dev_glove_embedding, dev_glove_vocab = create_glove_embedding(
        vocab=dev_vocab, 
        glove_base_filename="glove.42B.300d.txt",
    )
    dev_glove_vocab += [UNK_SYMBOL]

    # validation embeddings
    val_glove_embedding, val_glove_vocab = create_glove_embedding(
        vocab=sorted({w for toks in dev_seqs_train + dev_seqs_test for w in toks}), 
        glove_base_filename="glove.42B.300d.txt",
    )

    val_glove_vocab += [UNK_SYMBOL]

In [367]:
# PLEASE MAKE SURE TO INCLUDE THE FOLLOWING BETWEEN THE START AND STOP COMMENTS:
#   1) Textual description of your system.
#   2) The code for your original system.
#   3) The score achieved by your system in place of MY_NUMBER.
#        With no other changes to that line.
#        You should report your score as a decimal value <=1.0
# PLEASE MAKE SURE NOT TO DELETE OR EDIT THE START AND STOP COMMENTS

# NOTE: MODULES, CODE AND DATASETS REQUIRED FOR YOUR ORIGINAL SYSTEM
# SHOULD BE ADDED BELOW THE 'IS_GRADESCOPE_ENV' CHECK CONDITION. DOING
# SO ABOVE THE CHECK MAY CAUSE THE AUTOGRADER TO FAIL.

# START COMMENT: Enter your system description in this cell.
# My peak score was: MY_NUMBER

# idea 1: lstm cells, target color vector concat as above, 300d glove embedding, grid search
# idea 2: embeddings from BERT? (def get_bert_embeddings() -> vocab, np.matrix)
# idea 3: higher hidden dimensions in LSTM & more layers.

class OriginalEncoder(Encoder):
    def __init__(self, *args, num_layers=2, rnn_type=nn.GRU, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_layers = num_layers
        self.rnn = rnn_type(
            input_size=self.color_dim,
            hidden_size=self.hidden_dim,
            num_layers=self.num_layers,
            batch_first=True,
        )


class OriginalDecoder(Decoder):
    def __init__(self, *args, color_dim, num_layers=2, rnn_type=nn.GRU, **kwargs):
        self.color_dim = color_dim
        super().__init__(*args, **kwargs)
        self.num_layers = num_layers
        
        # input vectors dimension is the sum of 1D embedding vector
        # dimension and the 1D color vector dimension.
        self.input_size = self.embed_dim + self.color_dim
        self.rnn = rnn_type(
            input_size=self.input_size,
            hidden_size=self.hidden_dim,
            num_layers=self.num_layers,
            batch_first=True,
        )

    def get_embeddings(self, word_seqs, target_colors=None):
        word_only_embeddings = self.embedding(word_seqs)
        _, batch_len, _ = word_only_embeddings.shape  # no. of tokens in batch
        
        colors = target_colors.unsqueeze(1)  # convert each row to a column
        colors = torch.repeat_interleave(colors, repeats=batch_len, dim=1)
        return torch.cat(tensors=[word_only_embeddings, colors], dim=2)

class OriginalEncoderDecoder(EncoderDecoder):

    def forward(
        self,
        color_seqs,
        word_seqs,
        seq_lengths=None,
        hidden=None,
        targets=None,
    ):
        if hidden is None:
            hidden = self.encoder(color_seqs)

        # Extract the target colors from `color_seqs` and
        # feed them to the decoder, which already has a
        # `target_colors` keyword
        output, hidden = self.decoder(
            word_seqs, 
            seq_lengths=seq_lengths, 
            hidden=hidden, 
            # the last color is always the target color for all samples.
            target_colors=color_seqs[:, -1, :],  
        )

        # Your decoder will return `output, hidden` pairs; the
        # following will handle the two return situations that
        # the code needs to consider -- training and prediction.
        if self.training:
            return output
        else:
            return output, hidden


class OriginalContextualColorDescriber(ContextualColorDescriber):
    def __init__(self, *args, num_layers=2, rnn_type=nn.GRU, **kwargs):
        self.num_layers = num_layers
        self.rnn_type = rnn_type

        super().__init__(*args, **kwargs)
        self.params += ["num_layers", "rnn_type"]

    def build_graph(self):
        encoder = OriginalEncoder(
            color_dim=self.color_dim,
            hidden_dim=self.hidden_dim,
            num_layers=self.num_layers,
            rnn_type=self.rnn_type,
        )

        decoder = OriginalDecoder(
            vocab_size=self.vocab_size,
            embed_dim=self.embed_dim,
            color_dim=self.color_dim,
            embedding=self.embedding,
            hidden_dim=self.hidden_dim,
            num_layers=self.num_layers,
            rnn_type =self.rnn_type,
        )

        return OriginalEncoderDecoder(encoder, decoder)

def grid_search(color_seqs, word_seqs, base_mod):
    param_grid = {
        'hidden_dim': [10, 150, 500, 1200, 2000],
        "rnn_type": [nn.GRU, nn.LSTM],
        "num_layers": [2, 4, 8, 12],
        "freeze_embedding": [True, False],
    }
    search = RandomizedSearchCV(base_mod, param_grid, cv=2)
    search.fit(X=color_seqs, y=word_seqs)
    return search.best_estimator_

def test_original_system(use_toy=False):
    if use_toy:

        toy_color_seqs, toy_word_seqs, toy_vocab = create_example_dataset(
            group_size=50, 
            vec_dim=2,
        )
        toy_color_seqs_train, toy_color_seqs_test, toy_word_seqs_train, toy_word_seqs_test = \
            train_test_split(toy_color_seqs, toy_word_seqs)
        
        toy_mod = OriginalContextualColorDescriber(
            vocab=toy_vocab,
            embedding=dev_glove_embedding,
            early_stopping=True,
            embed_dim=300,
        )
        toy_mod = grid_search(toy_color_seqs_train, toy_word_seqs_train, toy_mod)

        _ = toy_mod.fit(toy_color_seqs_train, toy_word_seqs_train)
        acc = toy_mod.listener_accuracy(toy_color_seqs_test, toy_word_seqs_test)

        bleu_score, _ = toy_mod.corpus_bleu(toy_color_seqs_test, toy_word_seqs_test)
        print(
            "Listner Accuracy:\n", 
            toy_mod.listener_accuracy(toy_color_seqs_test, toy_word_seqs_test),
            "Bleu score:\n",
            bleu_score,
        )
    else:
        base_mod = OriginalContextualColorDescriber(
            vocab=dev_glove_vocab,
            embedding=dev_glove_embedding,
            early_stopping=True,
            embed_dim=300,
        )
        base_mod = grid_search(dev_cols_train, dev_seqs_train, base_mod)

        base_mod.fit(dev_cols_train, dev_seqs_train)
        acc = base_mod.listener_accuracy(dev_cols_test, dev_seqs_test)
        bleu_score, _ = base_mod.corpus_bleu(dev_cols_test, dev_seqs_test)
        print(
            "\nListner Accuracy:\n", 
            base_mod.listener_accuracy(dev_cols_test, dev_seqs_test),
            "\nBleu score:\n",
            bleu_score,
        )
    return acc


if 'IS_GRADESCOPE_ENV' not in os.environ:
    from sklearn.model_selection import RandomizedSearchCV
    import warnings
    
    warnings.filterwarnings("ignore")

    %time acc = test_original_system(use_toy=False)
    print(acc)

# STOP COMMENT: Please do not remove this comment.

Finished epoch 11 of 1000; error is 31.980353355407715

KeyboardInterrupt: 

## Bakeoff [1 point]

For the bake-off, we will use our original test set. The function you need to run for the submission is the following, which uses your `evaluate_original_system` from above:

In [ ]:
def create_bakeoff_submission(
    trained_model,
    output_filename='cs224u-colors-bakeoff-entry.csv',
):
    bakeoff_src_filename = os.path.join(
        "data", "colors", "cs224u-colors-test.csv")

    bakeoff_corpus = ColorsCorpusReader(bakeoff_src_filename)

    # This code just extracts the colors and texts from the new corpus:
    bakeoff_rawcols, bakeoff_texts = zip(*[
        [ex.colors, ex.contents] for ex in bakeoff_corpus.read()])

    # Original system function call; `trained_model` is your trained model:
    evaluation = evaluate_original_system(
        trained_model, bakeoff_rawcols, bakeoff_texts)

    evaluation['bakeoff_text'] = bakeoff_texts

    df = pd.DataFrame(evaluation)
    df.to_csv(output_filename)

In [ ]:
create_bakeoff_submission(dev_mod)

This creates a file `cs224u-colors-bakeoff-entry.csv` in the current directory. That file should be uploaded as-is. Please do not change its name.

Only one upload per team is permitted, and you should do no tuning of your system based on what you see in the file – you should not study that file in anyway, beyond perhaps checking that it contains what you expected it to contain. The upload function will do some additional checking to ensure that your file is well-formed.

The nature of our evaluation is such that we have to release the full test set with all labels. Thus, we have to trust you not to make any use of the test set during development. Recall:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.

Systems will be ranked primarily by `listener_accuracy`, but we will also consider their `corpus_bleu` scores. However, the BLEU score is just a simple check that your system is speaking some version of English that corresponds in some meaningful way to the gold descriptions, so you should concentrate on `listener_accuracy`.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points.